In [ ]:
import sys

#directory_]path = r'Working directory'

#sys.path.append(directory_path) # This path must contain this current file, mask_funcs.py and utils.py

In [1]:
import torch; print(torch.version.cuda); print(torch.cuda.is_available())

None
False


In [2]:
import torch
import numpy as np
import sys
import pandas as pd
import gc
import matplotlib.pyplot as plt
import os
from pathlib import Path
import datetime

import utils
import mask_funcs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from skimage.measure import label, regionprops, regionprops_table

In [4]:
from natsort import natsorted

In [5]:
from re import L
class Cell:
    def __init__(self, index, directory):
        self.index = index
        self.file = directory / ("{0:04}".format(self.index) + '.csv')
        with open(self.file, 'w') as f:
            f.write('time\tarea\tspeed\tperimeter\txcentre\tycentre\teccentricity\torientation\tmajor_axis\tminor_axis')

    def write_features(self, line):
        with open(self.file, 'a') as f:
            f.write(line)

    def save_pd(self):
      pass

class CellBatch:
    def __init__(self, indices, tracked_directory,  features_directory):
        self.indices = indices
        self.expanded_indices = self.indices.unsqueeze(-1).unsqueeze(-1).expand((len(indices), *IMAGE_SIZE))

        self.cells = [Cell(i, features_directory) for i in self.indices]
        self.centres = None

        self.timer = '00:00:00.0'
        self.timer = datetime.datetime.strptime(self.timer, '%H:%M:%S.%f')
    
        self.last_centres = None
        self.batch_size = len(self.indices)
        self.paths = natsorted([p for p in (tracked_directory).iterdir()])
       
        self.num_frames = len(self.paths)
        self.coord_grid_x, self.coord_grid_y = torch.meshgrid(torch.arange(IMAGE_SIZE[0]).to(device),
                                                              torch.arange(IMAGE_SIZE[1]).to(device))

        self.tracked_directory = tracked_directory
        

    def run_feature_extraction(self):
        for i, path in enumerate(self.paths):
            print(self.paths)
            sys.stdout.write(f'\rFrame {i+1} | Cells {torch.min(self.indices)}-{torch.max(self.indices)} \n')
            sys.stdout.flush()
            if i == 0:
                full_mask = torch.tensor(utils.read_tiff(path).astype(np.int16)).to(device)
                print(full_mask.max())
                self.masks = torch.where(full_mask.unsqueeze(0).expand(len(self.indices), *full_mask.shape) == self.expanded_indices, 1,0)
                print(self.masks[0].min())
                full_mask = None
            self.next_frame(path)
            self.read_features()
            
            self.write_features()
            torch.cuda.empty_cache()
            gc.collect()

    def next_frame(self, path):
        full_mask = torch.tensor(utils.read_tiff(path).astype(np.int16)).to(device)
        self.masks = torch.where(full_mask.unsqueeze(0).expand(len(self.indices), *full_mask.shape) == self.expanded_indices, 1, 0)
        full_mask = None
    

    def read_features(self):
        self.get_time()
        self.get_areas()
        self.get_centres()
        self.get_ecc()
        self.get_ori()
        self.get_axs()
        self.get_speeds()
        self.get_perimeters()
        

    def write_features(self):
        for i, cell in enumerate(self.cells):
            new_line = (
                '\n'  + str(self.timer.strftime('%M:%S')) + '\t'
                + '\t'.join([str(a.item()) for a in (self.areas[i], self.speeds[i], self.perimeters[i], self.centres[i, 0], self.centres[i, 1])])
                + '\t' + str(self.ecc[i])
                + '\t' + str(self.ori[i])
                + '\t' + str(self.maj[i])
                + '\t' + str(self.min[i])
            )
            
            cell.write_features(new_line)
        self.timer += datetime.timedelta(seconds=3)   
        
    def get_time(self):
        self.times = []
        
    def get_areas(self):
        self.areas = torch.sum(self.masks, dim=(1, 2)).float()
        self.areas[self.areas == 0] = float('nan')

    def get_centres(self):
        if self.centres is not None:
            self.last_centres = self.centres

        x_centres = torch.sum(self.masks * self.coord_grid_x, dim=(1, 2)) / self.areas
        y_centres = torch.sum(self.masks * self.coord_grid_y, dim=(1, 2)) / self.areas

        self.centres = torch.stack((x_centres, y_centres), dim=1)

    def get_ecc(self):
      self.ecc = []
      for am in self.masks:
        props = regionprops_table(
      am.cpu().numpy().astype(np.uint8),
      properties=('eccentricity',),
                                )
        if len(props['eccentricity']) == 0:
          self.ecc.append(float('nan'))
        else:
          self.ecc.append(props['eccentricity'][0])

    def get_ori(self):
      self.ori = []
      for am in self.masks:
        props = regionprops_table(
      am.cpu().numpy().astype(np.uint8),
      properties=('orientation',),
                                )
        if len(props['orientation']) == 0:
          self.ori.append(float('nan'))
        else:
          self.ori.append(props['orientation'][0])

    def get_axs(self):
      self.maj = []
      self.min = []
      for am in self.masks:
        props = regionprops_table(
      am.cpu().numpy().astype(np.uint8),
      properties=('axis_major_length','axis_minor_length',),
                                )
        if len(props['axis_major_length']) == 0:
          self.maj.append(float('nan'))
        else:
          self.maj.append(props['axis_major_length'][0])

        if len(props['axis_minor_length']) == 0:
          self.min.append(float('nan'))
        else:
          self.min.append(props['axis_minor_length'][0])

    def get_speeds(self):
        if self.last_centres is None:
            self.speeds = torch.full((self.batch_size,), float('nan'))

        else:
            self.speeds = torch.sqrt((self.centres[:, 0] - self.last_centres[:, 0])**2 + (self.centres[:, 1] - self.last_centres[:, 1])**2)
        del self.last_centres

    def get_perimeters(self):
        kernel = torch.tensor([[1, 1, 1],
                               [1, 9, 1],
                               [1, 1, 1]]).to(device)

        padded_masks = torch.nn.functional.pad(self.masks, (1, 1, 1, 1), mode='constant', value=0)
        conv_result = torch.nn.functional.conv2d(padded_masks.unsqueeze(1).float(), kernel.unsqueeze(0).unsqueeze(0).float(),
                                                 padding=0).squeeze()
        self.perimeters = torch.sum((conv_result >= 10) & (conv_result <=16), dim=(1, 2)).float()
        self.perimeters[self.perimeters == 0] = float('nan')


    def clean_up(self, threshold=50):
      
      self.tracked_masks = sorted([mask for mask in Path(phase_directory).iterdir()])
      
      length_of_tracks = {}
      for i, frame_path in enumerate(self.tracked_masks):
          sys.stdout.write(
              f'\rReading frame {i + 1} / {len(self.tracked_masks)}')
          sys.stdout.flush()
          frame = torch.tensor(utils.read_tiff(frame_path).astype(np.int16)).cpu()
          for index in torch.unique(frame):
              index = index.item()
              if index != 0:
                  if index not in length_of_tracks.keys():
                      length_of_tracks[index] = 0
                  length_of_tracks[index] += 1
      tracks_to_remove = torch.tensor(
          [index for index, track_length in length_of_tracks.items() if track_length < threshold]).cpu()
      print('\n',length_of_tracks)
      print(tracks_to_remove)
      for i, frame_path in enumerate(self.tracked_masks):
          # BATCHES NEEDED TO SPEED THIS BIT UP
          sys.stdout.write(
              f'\rCleaning frame {i + 1} / {len(self.tracked_masks)}')
          sys.stdout.flush()
          frame = torch.tensor(utils.read_tiff(frame_path).astype(np.int16)).cpu()
          cleaned_frame = frame.clone()
          for track in tracks_to_remove:
              cleaned_frame[frame == track] = 0
          utils.save_tiff(cleaned_frame.to(dtype=torch.int16).cpu().numpy().astype(np.uint16), frame_path)


## Feature Extraction
For each detected cell, saves .csv file with value for each feature at each frame

In [6]:

# inputs
tracked_directory = r'Tracks'

# output
features_directory = r'Features'
phase_directory = r'Phase_masks/amoeba'

utils.remake_dir(Path(features_directory))

IMAGE_SIZE = utils.read_tiff([i for i in (Path(tracked_directory).iterdir())][0]).shape

with torch.no_grad():
    cell_batch = CellBatch(torch.tensor(np.arange(1, 101)).to(device), Path(tracked_directory),  Path(features_directory))
    cell_batch.run_feature_extraction()

[WindowsPath('Tracks/0000.tif'), WindowsPath('Tracks/0001.tif'), WindowsPath('Tracks/0002.tif'), WindowsPath('Tracks/0003.tif'), WindowsPath('Tracks/0004.tif'), WindowsPath('Tracks/0005.tif'), WindowsPath('Tracks/0006.tif'), WindowsPath('Tracks/0007.tif'), WindowsPath('Tracks/0008.tif'), WindowsPath('Tracks/0009.tif'), WindowsPath('Tracks/0010.tif'), WindowsPath('Tracks/0011.tif'), WindowsPath('Tracks/0012.tif'), WindowsPath('Tracks/0013.tif'), WindowsPath('Tracks/0014.tif'), WindowsPath('Tracks/0015.tif'), WindowsPath('Tracks/0016.tif'), WindowsPath('Tracks/0017.tif'), WindowsPath('Tracks/0018.tif'), WindowsPath('Tracks/0019.tif'), WindowsPath('Tracks/0020.tif'), WindowsPath('Tracks/0021.tif'), WindowsPath('Tracks/0022.tif'), WindowsPath('Tracks/0023.tif'), WindowsPath('Tracks/0024.tif'), WindowsPath('Tracks/0025.tif'), WindowsPath('Tracks/0026.tif'), WindowsPath('Tracks/0027.tif'), WindowsPath('Tracks/0028.tif'), WindowsPath('Tracks/0029.tif'), WindowsPath('Tracks/0030.tif'), Windows

C:\Users\2MY\anaconda3\envs\Research\Lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor(51, dtype=torch.int16)
tensor(0)
[WindowsPath('Tracks/0000.tif'), WindowsPath('Tracks/0001.tif'), WindowsPath('Tracks/0002.tif'), WindowsPath('Tracks/0003.tif'), WindowsPath('Tracks/0004.tif'), WindowsPath('Tracks/0005.tif'), WindowsPath('Tracks/0006.tif'), WindowsPath('Tracks/0007.tif'), WindowsPath('Tracks/0008.tif'), WindowsPath('Tracks/0009.tif'), WindowsPath('Tracks/0010.tif'), WindowsPath('Tracks/0011.tif'), WindowsPath('Tracks/0012.tif'), WindowsPath('Tracks/0013.tif'), WindowsPath('Tracks/0014.tif'), WindowsPath('Tracks/0015.tif'), WindowsPath('Tracks/0016.tif'), WindowsPath('Tracks/0017.tif'), WindowsPath('Tracks/0018.tif'), WindowsPath('Tracks/0019.tif'), WindowsPath('Tracks/0020.tif'), WindowsPath('Tracks/0021.tif'), WindowsPath('Tracks/0022.tif'), WindowsPath('Tracks/0023.tif'), WindowsPath('Tracks/0024.tif'), WindowsPath('Tracks/0025.tif'), WindowsPath('Tracks/0026.tif'), WindowsPath('Tracks/0027.tif'), WindowsPath('Tracks/0028.tif'), WindowsPath('Tracks/0029.tif'),

In [7]:
cell_batch.clean_up()

Reading frame 64 / 64
 {1: 64, 2: 64, 3: 64, 4: 64, 5: 64, 6: 64, 7: 64, 8: 64, 9: 64, 10: 64, 11: 64, 12: 64, 13: 64, 14: 64, 15: 64, 16: 64, 17: 64, 18: 64, 19: 64, 20: 64, 21: 64, 22: 64, 23: 64, 24: 64, 25: 64, 26: 64, 27: 64, 28: 64, 29: 64, 30: 64, 31: 64, 32: 64, 33: 64, 34: 64, 35: 64, 36: 64, 37: 64, 38: 63, 39: 64, 40: 61, 41: 62, 42: 63, 43: 62, 44: 59, 45: 59, 46: 55, 47: 56, 48: 51, 50: 48, 51: 47, 49: 48, 52: 47, 53: 40, 54: 38, 55: 37, 56: 34, 58: 30, 59: 25, 60: 20, 61: 18, 57: 25, 62: 15, 63: 11, 64: 7, 65: 5, 66: 3, 67: 3, 68: 1}
tensor([50, 51, 49, 52, 53, 54, 55, 56, 58, 59, 60, 61, 57, 62, 63, 64, 65, 66,
        67, 68])
Cleaning frame 64 / 64

In [8]:
def clean_directory(features_directory,out_dir):
    utils.remake_dir(Path(out_dir))

    for i,cell in enumerate((Path(features_directory)).iterdir()):
      df = pd.read_csv(cell, sep='\t')
    
      if np.isnan(df['area'][0]) == True:
        continue
      else:
          filepath = os.path.join(out_dir,f'cell_{i+1}')
          df.to_csv(f'{filepath}.csv', index=False)

viable_cell_dir = r'final_cells'
clean_directory(features_directory, viable_cell_dir)